In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ler.rates import LeR
from scipy.stats import norm
from astropy.cosmology import LambdaCDM
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)
from scipy.interpolate import interp1d
import os
import contextlib
from ler.utils import load_json, append_json, get_param_from_json, batch_handler

In [2]:
ler = LeR(verbose=False)

In [3]:
ler.batch_size = 1000
lensed_params = ler.lensed_cbc_statistics(size=1000)

chosen batch size = 1000 with total size = 1000
There will be 1 batche(s)
Batch no. 1
sampling lensed params...
solving lens equations...


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 264.57it/s]


calculating snrs...


In [6]:
ler.lensed_rate();

getting lensed_params from json file ./lensed_param.json...
total lensed rate (yr^-1) (with step function): 8.946125164990804
storing detectable lensed params in ./lensed_param_detectable.json


## Data generation

* find a range for redshift

In [13]:
def find_std(mean, lower90, upper90):
    """
    Function to find the standard deviation of a normal distribution
    """
    # Z-scores for 5% and 95% of the standard normal distribution
    z_lower = norm.ppf(0.05)
    z_upper = norm.ppf(0.95)

    # Solving for standard deviation (σ)
    std_dev_lower = (lower90 - mean) / z_lower
    std_dev_upper = (upper90 - mean) / z_upper

    # The standard deviation should be the same for both calculations
    std_dev = (std_dev_lower + std_dev_upper) / 2

    return std_dev

In [7]:
# for coversion, from luminosity distance to redshift
z = np.linspace(0, 10, 1000)
luminosity_distance = cosmo.luminosity_distance(z).value
luminosity_distance_to_z = interp1d(luminosity_distance, z, kind='cubic', fill_value='extrapolate')
z_to_luminosity_distance = interp1d(z, luminosity_distance, kind='cubic', fill_value='extrapolate')

In [11]:
# this range for 90% credible interval, is good enough 
z_to_luminosity_distance(0.5)-z_to_luminosity_distance(0.3)

1280.2221632664455

* find standard deviation of redshift

In [15]:
print("for redshift: ", find_std(0.4,0.3,0.5))
print("for chirp_mass: ", find_std(45,50,60))

for redshift:  0.06079568319117691
for chirp_mass:  3.0397841595588466


In [16]:
# choosen parameters
chirp_mass_data = np.random.normal(45, 3.0397841595588466, 1000)
zs_eff_data = 0.5 + np.random.normal(0.4, 0.06079568319117691, 1000)

## Considering the corellation between Mc and zs_eff

In [ ]:
zs_eff_vs_chirp_mass=dict(x=zs_eff, y=mc_lensed_source, x_kde=zs_unlensed, y_kde=mc_unlensed_source, percentile=90, region='outside')

In [ ]:
# ["optimal_snr_net", "zs-chirp_mass", "sky_location", "time_delay"
def selecting_n_lensed_detectable_events(
    size=100,
    batch_size=None,
    resume=False,
    output_jsonfile="./lensed_params_detectable.json",
    selection_params=dict(
        optimal_snr_net=dict(snr_threshold=8.0, num_img=2),
        zs_eff_vs_chirp_mass=dict(x=zs_eff, y=mc_lensed_source, x_kde=zs_unlensed, y_kde=mc_unlensed_source, percentile=90, region='outside'),
        ),
    ),
):
    """
    Function to select n lensed detectable events.

    Parameters
    ----------
    size : int
        Number of lensed detectable events to be selected.
    batch_size : int
        Number of lensed events to be generated in each batch.
    resume : bool
        Resume the selection from the last saved json file.
    output_jsonfile : str
        Path to the output json file.
    selection_params : dict
        Dictionary containing the selection parameters. The keys are the names of the parameters wrt which the selection will be done and the values are either the parameters for each selection method or  both such selection paramerters and the posterior distribution of the gw parameter. 
    """
    
    if batch_size is None:
        batch_size = ler.batch_size

    if not resume:
        n = 0  # iterator
        try:
            os.remove(output_jsonfile)
        except:
            pass
    else:
        # get sample size as size from json file
        param_final = get_param_from_json(output_jsonfile)
        n = len(param_final["zs"])
        del param_final

    # sorting condition 
    snr_threshold = selection_params["optimal_snr_net"]["snr_threshold"]
    num_img = selection_params["optimal_snr_net"]["num_img"]

    # sorting zs_eff vs chirp_mass condition
    zs_unlensed_ = selection_params["zs_eff_vs_chirp_mass"]["x_kde"]
    mc_unlensed_source_ = selection_params["zs_eff_vs_chirp_mass"]["y_kde"]
    xy = np.vstack([zs_unlensed_, mc_unlensed_source_])
    kernal = gaussian_kde(xy)
    kde = kernal(xy)
    idx = kde.argsort()
    z = kde[idx]
    
    buffer_file = "./lensed_params_buffer.json"
    print("collected number of events = ", n)
    while n < size:
        # disable print statements
        with contextlib.redirect_stdout(None):
            ler.lensed_sampling_routine(
                size=ler.batch_size, output_jsonfile=buffer_file, resume=False
            )

            # get all lensed events
            # Dimensions are (size, n_max_images)
            lensed_param = get_param_from_json(buffer_file)

            # event selection wrt snr

            # event selection wrt zs_eff vs chirp_mass


            # store all params in json file
            for key, value in lensed_param.items():
                lensed_param[key] = np.nan_to_num(value[snr_hit])
            append_json(output_jsonfile, lensed_param, replace=False)

            n += np.sum(snr_hit)
        print("collected number of events = ", n)

    # trim the final param dictionary
    print(f"trmming final result to size={size}")
    param_final = get_param_from_json(output_jsonfile)
    # trim the final param dictionary
    idx = np.random.choice(len(param_final["zs"]), size, replace=False)
    for key, value in param_final.items():
        param_final[key] = param_final[key][idx]

    # save the final param dictionary
    append_json(output_jsonfile, param_final, replace=True)

    return param_final